In [ ]:
#%run ../input/minivgg2/MiniVGG11/build_hdf5.py

In [ ]:
import os
os.remove("./train.hdf5") 
os.remove("./test.hdf5") 
os.remove("./VAL.hdf5") 


In [ ]:
#%run ../input/minivgg2/MiniVGG11/mini_vgg_11_train.py

In [ ]:
#%run ../input/minivgg2/MiniVGG11/evaluating.py

In [ ]:
#样本文件路径
SAMPLE_FILE_PATH = "../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv"
#分类数量
NUM_CLASSES = 7
#训练、校验、测试数据集HDF5文件的输出路径
TRAIN_HDF5 = "/kaggle/working/train.hdf5"
VAL_HDF5 = "/kaggle/working/VAL.hdf5"
TEST_HDF5 = "/kaggle/working/test.hdf5"

#每批次样本数量
BATCH_SIZE = 128

#项目输出文件保存目录
OUTPUT_PATH = "/kaggle/working/"

#数据集样本RGB平均值存储位置及文件名称
DATASET_MEAN_FILE = OUTPUT_PATH + "rgb_mean.json"

#模型保存位置及文件名称
MODEL_FILE = OUTPUT_PATH + "model.h5"

In [ ]:
from tensorflow.keras.callbacks import Callback
import os
class EpochCheckpoint(Callback):
    def __init__(self, output_path, every=5, start_at=0):
        super(Callback, self).__init__()
        self.output_path = output_path
        self.every = every
        self.start_epoch = start_at
    def on_epoch_end(self, epoch, logs={}):
        self.start_epoch += 1

In [ ]:
import h5py
import numpy as np
# from tensorflow.keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
class HDF5DatasetGenerator:
    def __init__(self, db_file, batch_size, preprocessors=None, aug=None, binarize=True, classes=2):
        self.batchSize = batch_size
        self.preprocessors = preprocessors
        self.aug = aug
        self.binarize = binarize
        self.classes = classes
        self.db = h5py.File(db_file)
        self.numImages = self.db["labels"].shape[0]
    def generator(self, passes=np.inf):
        epochs = 0
        while epochs < passes:
            for i in np.arange(0, self.numImages, self.batchSize):
                images = self.db["images"][i:i + self.batchSize]
                labels = self.db["labels"][i:i + self.batchSize]
                if self.binarize:
                    labels = to_categorical(labels, self.classes)
                if self.preprocessors is not None:
                    processed_images = []
                    for image in images:
                        for p in self.preprocessors:
                            image = p.preprocess(image)
                        processed_images.append(image)
                    images = np.array(processed_images)
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images, labels, batch_size=self.batchSize))
                yield images, labels
            epochs += 1
            if epochs==5000:
                break
    def close(self):
        self.db.close()

HDF5文件数据集生成器

In [ ]:
import os
import h5py
# HDF5数据集生成器
class HDF5DatasetWriter:
    def __init__(self, dims, output_path, data_key="images", buf_size=1000):
        if os.path.exists(output_path):
            raise ValueError("您提供的输出文件{}已经存在，请先手工删除！".format(output_path))
        self.db = h5py.File(output_path, "w")
        self.data = self.db.create_dataset(data_key, dims, dtype="float")
        self.labels = self.db.create_dataset("labels", (dims[0],), dtype="int")
        self.buf_size = buf_size  # 1000
        self.buffer = {"data": [], "labels": []}
        self.idx = 0
    def add(self, raw, label):
        self.buffer["data"].extend(raw)
        self.buffer["labels"].extend(label)
        if len(self.buffer["data"]) >= self.buf_size:
            self.flush()
    def flush(self):
        i = self.idx + len(self.buffer["data"])
        self.data[self.idx:i] = self.buffer["data"]
        self.labels[self.idx:i] = self.buffer["labels"]
        self.idx = i
        self.buffer = {"data": [], "labels": []}
    def store_class_labels(self, class_labels):
        dt = h5py.special_dtype(vlen=str)
        label_dim = (len(class_labels),)
        label_set = self.db.create_dataset("label_names", label_dim, dtype=dt)
        label_set[:] = class_labels
    def close(self):
        if len(self.buffer["data"]) > 0:
            self.flush()
        self.db.close()

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
class ImageToArrayPreprocessor:
    def __init__(self, data_format=None):
        self.data_format = data_format
    def preprocess(self, image):
        return img_to_array(image, data_format=self.data_format)

训练绘图

In [ ]:
import json
import os
import numpy as np
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
class TrainingMonitor(BaseLogger):
    def __init__(self, fig_path, json_path=None, start_at=0):
        super(TrainingMonitor, self).__init__()
        self.fig_path = fig_path
        self.json_path = json_path
        self.start_at = start_at
    def on_train_begin(self, logs={}):
        self.history = {}
        if self.json_path is not None:
            if os.path.exists(self.json_path):
                self.history = json.loads(open(self.json_path).read())
                if self.start_at > 0:
                    for k in self.history.keys():
                        self.history[k] = self.history[k][:self.start_at]
    def on_epoch_end(self, epoch, logs={}):
        for (k, v) in logs.items():
            log = self.history.get(k, [])
            log.append(float(v))
            self.history[k] = log
        if self.json_path is not None:
            f = open(self.json_path, 'w')
            f.write(json.dumps(self.history, skipkeys=True))
            f.close()
        if len(self.history["loss"]) > 1:
            N = np.arange(0, len(self.history["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.history["loss"], label="train_loss")
            plt.plot(N, self.history["val_loss"], label="val_loss")
            plt.plot(N, self.history["accuracy"], label="train_acc")
            plt.plot(N, self.history["val_accuracy"], label="val_acc")
            epochs = len(self.history["loss"])
            plt.title("Training Loss & Accuracy [Epoch {}]".format(epochs))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            plt.savefig(self.fig_path)
            plt.close()

hdf5数据的生成

In [ ]:
import numpy as np
print('加载csv格式数据集文件...')

file=open(SAMPLE_FILE_PATH)
# 跳过表头
file.__next__()
(train_images,train_labels)=([],[])
(val_images,val_labels)=([],[])
(test_images,test_labels)=([],[])
# 存储数据集样本类别分布的字典

count_by_label_train={}
count_by_label_val={}
count_by_label_test={}

# 遍历一遍文件的每一行
for row in file:
    (label,usage,image)=row.strip().split(",")
    # 标签转化整数
    label=int(label)
    image=np.array(image.split(' '),dtype='uint8')
    image=image.reshape((48,48))
    #用途为train，添加训练样本数据集
    if usage=='Training':
        train_images.append(image)
        train_labels.append(label)
        count=count_by_label_train.get(label,0)
        count_by_label_train[label]=count+1
    # 用 途PubilcTest，添加到校验样本集
    elif usage=='PublicTest':
        val_images.append(image)
        val_labels.append(label)
        count=count_by_label_val.get(label,0)
        count_by_label_val[label]=count+1
    # 用 PrivateTest，添加到测试样本集
    elif usage=='PrivateTest':
        test_images.append(image)
        test_labels.append(label)
        count=count_by_label_test.get(label,0)
        count_by_label_test[label]=count+1
file.close()

print("训练样本数量:{}".format(len(train_images)))
print("校验样本数量:{}".format(len(val_images)))
print("测试样本数量:{}".format(len(test_images)))
# 输出训练、校验、测试样本分布
print(count_by_label_train)
print('校验样本分布')
print(count_by_label_val)
print("测试样本分布")
print(count_by_label_test)
# 构建一个训练、校验、测试数据集列表
# 每个元素由数据集类型名称、全部样本文件名称、全部样本整形标签、HDF5输出文件构成
datasets=[(train_images,train_labels,TRAIN_HDF5),
          (val_images,val_labels,VAL_HDF5),
          (test_images,test_labels,TEST_HDF5)]

# 遍历数据集元祖
for (images,labels,outputPath) in datasets:
    #     创建HDF5写入器
    print("构建 {}...".format(outputPath))
    writer=HDF5DatasetWriter((len(images),48,48),outputPath)
    #遍历每个图像，将其加入数据集
    for (image,label) in zip(images
                             ,labels):
        writer.add([image],[label])

    #关闭HDF5写入器
    writer.close()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras import backend
class MiniVGG11:
    @staticmethod
    def build(width,height,channel,classes,reg=0.0002):
        """
        Args:
            width:输入样本的宽度
            height: 输入样本的长度
            channel: 输入样本的通道
            classes: 分类数量
            reg: 正则化因子
        Returns:  MiniVGG11 网络模型对象
        """
        model=Sequential(name='MiniVGG11')
        shape=(height,width,channel)
        channel_dimension=-1
        if backend.image_data_format()=='channels_first':
            shape=(channel,height,width)

        # 如果输入格式为通道前置
        #重新设置输入格式和通道位置指示
        if backend.image_data_format()=='channels_first':
            shape=(channel,height,width)
            channel_dimension=1
            # 第一卷积快
        model.add(Conv2D(64, (3, 3), input_shape=shape, padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # 第二卷积快
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # 第三卷积快
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))

        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # 第四卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))

        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # 第五卷积快
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))

        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same", strides=(1, 1)))
        # 第一全连接
        model.add(Flatten())
        model.add(Dense(256, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        # 第二全练接
        model.add(Dense(128, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        # 第三全连接
        model.add(Dense(classes, kernel_regularizer=l1(reg)))
        model.add(Activation("softmax"))
        return model

if __name__=='__main__':
    model=MiniVGG11.build(width=48,height=48,channel=1,classes=7,reg=0.0002)
    print(model.summary())

minivgg11模型训练

In [ ]:
import matplotlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

import argparse
import json
import os

matplotlib.use("Agg")

parser = argparse.ArgumentParser()
parser.add_argument("-c", "--checkpoints", help="检查点输出目录")

parser.add_argument("-m", "--model", type=str, help="要加载的检查点模型文件")
parser.add_argument("-s", "--start-epoch", type=int, default=0, help="重新训练的趟数期起点")

args = vars(parser.parse_args(args=[]))



train_aug = ImageDataGenerator(rotation_range=10,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               rescale=1 / 255.0,
                               fill_mode="nearest")

val_aug = ImageDataGenerator(rescale=1 /255.0)

iap = ImageToArrayPreprocessor()

train_gen = HDF5DatasetGenerator(TRAIN_HDF5,
                                 BATCH_SIZE,
                                 aug=train_aug,
                                 preprocessors=[iap],
                                 classes=NUM_CLASSES)

val_gen = HDF5DatasetGenerator(VAL_HDF5,
                                 BATCH_SIZE,
                                 aug=val_aug,
                                 preprocessors=[iap],
                                 classes=NUM_CLASSES)

if args["model"] is None:
    print("[信息] 编译模型。。。")
    opt = Adam(lr=1e-3)
    model = MiniVGG11.build(width=48, height=48, channel=1, classes=NUM_CLASSES)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics = ["accuracy"])
else:
    print("[信息] 加载{}。。。".format(args["model"]))
    model = load_model(args["model"])
    print("[信息] 原学习率：{}。。。".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-3)
    print("[信息] 现学习率：{}。。。".format(K.get_value(model.optimizer.lr)))
fig_path = os.path.sep.join([OUTPUT_PATH, "MiniVGG11.png"])
json_path = os.path.sep.join([OUTPUT_PATH, "MiniVGG11.json"])
callbacks = [EpochCheckpoint(args["checkpoints"],every=5, start_at=args["start_epoch"]),
             TrainingMonitor(fig_path=fig_path, json_path=json_path, start_at=args["start_epoch"])
]
model.fit_generator(train_gen.generator(),
                    steps_per_epoch = train_gen.numImages//BATCH_SIZE,
                    validation_data = val_gen.generator(),
                    validation_steps = val_gen.numImages//BATCH_SIZE,
                    epochs=30,
                    max_queue_size= BATCH_SIZE *2,
                    callbacks=callbacks,
                    verbose=1)

print("[信息] 保存模型。。。")
model.save(MODEL_FILE, overwrite=True)

train_gen.close()
val_gen.close() 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
teat_AUG = ImageDataGenerator(rescale=1 / 255.0)
iap = ImageToArrayPreprocessor()
testGen = HDF5DatasetGenerator(TEST_HDF5,
                               BATCH_SIZE,
                               aug=teat_AUG,
                               preprocessors=[iap],
                               classes=NUM_CLASSES)
print("[信息]加载网络模型....")
model = load_model(MODEL_FILE)
(loss, acc) = model.evaluate_generator(testGen.generator(),
                                       steps=testGen.numImages //BATCH_SIZE,
                                       max_queue_size=BATCH_SIZE * 2)
print("[信息]测试集准确率: {:.2f}%".format(acc * 100))
testGen.close()